In [1]:
!pip install change-detection-pytorch

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 220.1 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12403 sha256=18dedc94c7a0ca6c03e3f92f6be4cb36d77557824b093f2bdad799bde9c5de48
  Stored in directory: /root/.cache/pip/wheels/2d/3f/1a/183399ebaa5998d3e4d60780ca5951bbeab4e6b69cc3fea227
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=8a1cc87535d8459354c43fea6f852a1340e26c4b232bfef773f65e46c0f76887
  Stored in directory: /root/.cache/pip/whe

In [2]:
import torch
from torch.utils.data import DataLoader, Dataset

import change_detection_pytorch as cdp
from change_detection_pytorch.datasets import CustomDataset, LEVIR_CD_Dataset, Dataset

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

model = cdp.Unet(
    encoder_name="resnet34",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,  # model output channels (number of classes in your datasets)
    siam_encoder=True,  # whether to use a siamese encoder
    fusion_form='concat',  # the form of fusing features from two branches. e.g. concat, sum, diff, or abs_diff.
)


train_dataset = LEVIR_CD_Dataset('data/train',
                                 sub_dir_1='time1',
                                 sub_dir_2='time2',
                                 img_suffix='.png',
                                 ann_dir='data/train/label',
                                 debug=False)

valid_dataset = LEVIR_CD_Dataset('data/val',
                                 sub_dir_1='time1',
                                 sub_dir_2='time2',
                                 img_suffix='.png',
                                 ann_dir='data/val/label',
                                 debug=False,
                                 test_mode=True)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=0)

loss = cdp.utils.losses.CrossEntropyLoss()
metrics = [
    cdp.utils.metrics.Precision(activation='argmax2d'),
    cdp.utils.metrics.Recall(activation='argmax2d'),
    cdp.utils.metrics.Fscore(activation='argmax2d'),
    cdp.utils.metrics.IoU(activation="argmax2d")
]

optimizer = torch.optim.Adam([
    dict(params=model.parameters(), lr=0.0001),
])

scheduler_steplr = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, ], gamma=0.1)

# create epoch runners
# it is a simple loop of iterating over dataloader`s samples
train_epoch = cdp.utils.train.TrainEpoch(
    model,
    loss=loss,
    metrics=metrics,
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = cdp.utils.train.ValidEpoch(
    model,
    loss=loss,
    metrics=metrics,
    device=DEVICE,
    verbose=True,
)

# train model for 60 epochs

max_score = 0
MAX_EPOCH = 10

for i in range(MAX_EPOCH):

    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    scheduler_steplr.step()

    # do something (save model, change lr, etc.)
    if max_score < valid_logs['fscore']:
        max_score = valid_logs['fscore']
        print('max_score', max_score)
        torch.save(model, './best_model.pth')
        print('Model saved!')

# save results (change maps)
"""
Note: if you use sliding window inference, set:
    from change_detection_pytorch.datasets.transforms.albu import (
        ChunkImage, ToTensorTest)

    test_transform = A.Compose([
        A.Normalize(),
        ChunkImage({window_size}}),
        ToTensorTest(),
    ], additional_targets={'image_2': 'image'})

"""
valid_epoch.infer_vis(valid_loader, save=True, slide=False, save_dir='res')

/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


Loaded 3800 images
Loaded 200 images

Epoch: 0
valid: 100%|██████████| 200/200 [00:05<00:00, 36.68it/s, cross_entropy_loss - 0.2817, fscore - 0.5273, precision - 0.6862, recall - 0.5211]
max_score 0.5272537768339336
Model saved!

Epoch: 1
valid: 100%|██████████| 200/200 [00:03<00:00, 54.14it/s, cross_entropy_loss - 0.2352, fscore - 0.5855, precision - 0.6811, recall - 0.6002]
max_score 0.585503879279907
Model saved!

Epoch: 2
valid: 100%|██████████| 200/200 [00:03<00:00, 53.96it/s, cross_entropy_loss - 0.2029, fscore - 0.5886, precision - 0.7853, recall - 0.5668]
max_score 0.5885864456110284
Model saved!

Epoch: 3
valid: 100%|██████████| 200/200 [00:03<00:00, 53.80it/s, cross_entropy_loss - 0.2077, fscore - 0.6106, precision - 0.7704, recall - 0.6145]
max_score 0.6105639670986875
Model saved!

Epoch: 4
valid: 100%|██████████| 200/200 [00:03<00:00, 54.11it/s, cross_entropy_loss - 0.1726, fscore - 0.641, precision - 0.7809, recall - 0.6505] 
max_score 0.6409697138680258
Model saved!

Epo